In [266]:
import pandas as pd
import numpy as np
import pprint

In [267]:
df = pd.read_csv('data.csv')

df

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31...40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31...40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [268]:
def entropy(target_column):
    values, counts = np.unique(target_column, return_counts=True)
    # print(values, counts)
    total_count = np.sum(counts)
    
    return -np.sum([(counts[i] / total_count) * np.log2(counts[i] / total_count) for i in range(len(counts))])


entropy(df['buys_computer'])

0.9402859586706311

In [269]:
def information_gain(df, split_attribute_column, target_column):
    data = df[split_attribute_column]
    # print(data)
    total_entropy = entropy(df[target_column])
    # print(total_entropy)

    values, counts = np.unique(data, return_counts=True)
    # print(values, counts)
    total_count = np.sum(counts)
    weighted_entropies = []
    for i in range(len(counts)):
        # print(f"weighted proportions for information gain {values[i]}: {counts[i]}/{total_count}")
        proportions = counts[i] / total_count
        split_df = df.where(df[split_attribute_column] == values[i]).dropna()
        # print(f"split df:")
        # print(split_df)
        split_df_target_column = split_df[target_column]
        # print("split df with labels")
        # print(split_df_target_column)
        weighted_entropy = proportions * entropy(split_df_target_column)
        weighted_entropies.append(weighted_entropy)

    return total_entropy - np.sum(weighted_entropies)
# information_gain(df, 'age', 'buys_computer')
features = df.columns[:-1]

for i in range(len(features)):
    gain = information_gain(df, features[i], "buys_computer")
    print({ features[i]: gain })

{'age': 0.24674981977443933}
{'income': 0.02922256565895487}
{'student': 0.15183550136234159}
{'credit_rating': 0.04812703040826949}


In [270]:
def get_best_feature_split(df, target_column):
    features = df.columns[:-1]
    information_gains = [information_gain(df, feature, target_column) for feature in features]
    highest_gain_index = np.argmax(information_gains)
    return features[highest_gain_index], information_gains[highest_gain_index] 

get_best_feature_split(df, 'buys_computer')

('age', 0.24674981977443933)

In [271]:
def build_tree(df: pd.DataFrame, target_column):

    values = np.unique(df[target_column])
    if (len(values) == 1):
        return values[0]
    
    best_feature, best_feature_gain = get_best_feature_split(df, target_column)

    tree = { best_feature: {} }

    for value in np.unique(df[best_feature]):
        subset = df.where(df[best_feature] == value).dropna()
        subset = subset.drop(columns=[best_feature])

        print(f"subset for {value}")
        print(f"{subset}")
        # Recursively create the subtree
        subtree = build_tree(subset, target_column)
        tree[best_feature][value] = subtree

    return tree

print(build_tree(df, 'buys_computer'))




subset for 31...40
    income student credit_rating buys_computer
2     high      no          fair           yes
6      low     yes     excellent           yes
11  medium      no     excellent           yes
12    high     yes          fair           yes
subset for <=30
    income student credit_rating buys_computer
0     high      no          fair            no
1     high      no     excellent            no
7   medium      no          fair            no
8      low     yes          fair           yes
10  medium     yes     excellent           yes
subset for no
   income credit_rating buys_computer
0    high          fair            no
1    high     excellent            no
7  medium          fair            no
subset for yes
    income credit_rating buys_computer
8      low          fair           yes
10  medium     excellent           yes
subset for >40
    income student credit_rating buys_computer
3   medium      no          fair           yes
4      low     yes          fair         

In [272]:
def predict(tree, sample):
    # Base case: if the tree is a leaf node (i.e., not a dictionary)
    if not isinstance(tree, dict):
        return tree
    print("in")
    # Get the root feature of the current tree
    root_feature = next(iter(tree))
    
    # Get the value of the root feature from the sample
    feature_value = sample[root_feature]

    # Check if the feature value exists in the tree
    if feature_value in tree[root_feature]:
        # Recur down to the corresponding subtree
        subtree = tree[root_feature][feature_value]
        return predict(subtree, sample)
    else:
        # Handle the case where the feature value does not exist in the tree
        return None  # or return a default value or the most common class
    
t = build_tree(df, 'buys_computer')
print()
print(predict(t, df.iloc[12].to_dict()))

subset for 31...40
    income student credit_rating buys_computer
2     high      no          fair           yes
6      low     yes     excellent           yes
11  medium      no     excellent           yes
12    high     yes          fair           yes
subset for <=30
    income student credit_rating buys_computer
0     high      no          fair            no
1     high      no     excellent            no
7   medium      no          fair            no
8      low     yes          fair           yes
10  medium     yes     excellent           yes
subset for no
   income credit_rating buys_computer
0    high          fair            no
1    high     excellent            no
7  medium          fair            no
subset for yes
    income credit_rating buys_computer
8      low          fair           yes
10  medium     excellent           yes
subset for >40
    income student credit_rating buys_computer
3   medium      no          fair           yes
4      low     yes          fair         